# Quantifying Pharmacy Deserts with Xarray-spatial

In this notebook, we'll demonstrate using Xarray-spatial to find and map the locations of pharmacy deserts in the USA.

### Setup:

First, we'll import some basic data and geospatial data-related packages. We'll also bring in some datashader functions for aggregation and rendering. And, finally, we'll import the Xarray-spatial functions we'll be using. 

In [ ]:
import datashader as ds
from datashader.colors import inferno
from datashader.transfer_functions import set_background
from datashader.transfer_functions import shade
from datashader.transfer_functions import stack
import geopandas as gpd
import numpy as np
import pandas as pd
from spatialpandas import GeoDataFrame

from xrspatial import proximity
from xrspatial.classify import binary
from xrspatial.classify import natural_breaks

## Load data

Let's load up the data we'll be playing with today: US pharmacy facilities, US block group data, and a US counties shapefile.

To download the examples data, run the command `xrspatial examples` in your terminal. All the data will be stored in your current directory inside a folder named `xrspatial-examples`.

First, we'll create a DataFrame of pharmacy facility locations and clean that up a bit and set the coordinates.

In [ ]:
# Load Pharmacies and add out x, y fields based on CalcLocation
pharmacy_df = pd.read_csv("./xrspatial-examples/data/facilities.csv")
coords = pharmacy_df["CalcLocation"].str.split(",", expand=True)
pharmacy_df["y"] = np.array(coords[0], dtype="float64")
pharmacy_df["x"] = np.array(coords[1], dtype="float64")
pharmacy_df

Next, we'll do the same for the US block group data, but we'll use geopandas for this, because it involves spatial data.
We'll also group together all age groups above 65 to simplify things a bit. 

In [ ]:
# Load Census Block Groups and Calculate Percent over 65 years-old
blockgroup_df = gpd.read_file("../xrspatial-examples/data/USA_Block_Groups.shp")
blockgroup_df = GeoDataFrame(blockgroup_df, geometry="geometry")
blockgroup_df["ABOVE_65"] = blockgroup_df[["AGE_65_74", "AGE_75_84", "AGE_85_UP"]].sum(
    axis=1
)
blockgroup_df["PCT_ABOVE_65"] = blockgroup_df["ABOVE_65"] / blockgroup_df["POP2010"]
blockgroup_df

And, we'll do the same again for the counties data.

In [ ]:
# Load Census County and Calculate Percent over 65 years-old
county_df = gpd.read_file("../xrspatial-examples/data/USA_Counties.shp")
county_df = GeoDataFrame(county_df, geometry="geometry")
county_df["ZONE_ID"] = county_df["OBJECTID"].astype(np.int16)
county_df

## Define Study Area and aggregate:

Now we're ready to set up our aggregate rasters (data that's been aggregated into the regular row-column cell format of a raster image) and then perform calculations on those rasters and combine them in informative ways.

The first raster we'll set up is the counties mask; we'll use this one to make our other rasters fit with county shapes.

- We set a range for for our x and y coordinates in longitude and latitude values.
- We use datashader's Canvas object as an easy-to-use frame to build up a raster with and aggregate data into.
- Finally, we aggregate the county shapes - polygons - into that raster. 

A quick shade and transformation with datashader functions visualizes our mask set-up.

In [ ]:
x_range = (-124.848974, -66.885444)
y_range = (24.396308, 49.384358)

W = 1600
H = 800

cvs = ds.Canvas(plot_width=W, plot_height=H, x_range=x_range, y_range=y_range)

county_mask = cvs.polygons(county_df, geometry="geometry")
set_background(shade(county_mask, cmap="#333333", alpha=255), "black")

Now we can set up the other rasters and use some Xarray-spatial functions to quantify pharmacy distances, age groups, and county data, and how they relate to each other.

### Create a "Distance to Nearest Pharmacy" Raster Layer & Classify into 4 Groups

First, we'll create a pharmacy locations raster layer by aggregating our pharmacies DataFrame from above.
Using `proximity`, we can create transform this into a raster of the distances for each point to its nearest pharmacy.
We can also further orgnaize this by applying Xarray-spatial's `natural_breaks` reclassification function. This will break up our proximity data into neat bins so it's easier to visualize and interpret.

In [ ]:
pharmacy_raster = cvs.points(pharmacy_df, "x", "y")
proximity_raster = proximity(pharmacy_raster, distance_metric="GREAT_CIRCLE").where(
    county_mask
)
proximity_raster.data[~np.isfinite(proximity_raster.data)] = 0.0

proximity_classifed = natural_breaks(proximity_raster, 20000, k=4).where(county_mask)

image_pharmacy = shade(proximity_classifed, cmap=inferno, alpha=255)
image_pharmacy = set_background(image_pharmacy, "black")
image_pharmacy

### Create an Age Layer  & Classify into 4 Groups

Next, we'll create an age groups raster from the block group DataFrame above.
We'll also reclassify this with `natural_breaks` to make it easier to map.

In [ ]:
age_raster = cvs.polygons(
    blockgroup_df, geometry="geometry", agg=ds.mean("PCT_ABOVE_65")
)
age_raster.data[~np.isfinite(age_raster.data)] = 0.0

age_classifed = natural_breaks(age_raster, 20000, k=4).where(county_mask)

age_image = shade(age_classifed, cmap=inferno, alpha=255)
age_image = set_background(age_image, "black")
age_image

### Combine layers to highlight seniors at risk from pharmacy deserts

Finally, we'll combine the county, age, and pharmacy distance raster layers to highlight counties where there are more seniors who are at greater distance from pharmacies.

In [ ]:
pharmacy_deserts = binary(proximity_classifed, [3])
older_regions = binary(age_classifed, [3])
target_deserts = (pharmacy_deserts * older_regions).where(county_mask)
target_deserts_img = shade(
    target_deserts, cmap=["#333333", "fuchsia"], alpha=255, how="linear"
)
set_background(target_deserts_img, "black")

###  Summarize seniors at risk by county: the Zonal Statistics function

The renderings above are great, but we might also want to summarize the quantitative info we just generated as a table with summary statistics. Xarray-spatial's zonal statisitics function lets us do that. 

`Zonal statistics` operates on a values raster by applying a zones aggregate to it. This is defined by creating an integer aggregate corresponding to the shape of the values raster, but with the value set at each cell to a non-zero integer that is the id of the zone we wish to put that cell in. `Zonal_stats` gathers the values for all cells assigned to each zone_id and generates summary statistics for each one. The output is a Pandas dataframe with the calculation results in columns and with a row for each zone.

#### Setup:

First, we'll set up our counties raster as a background for the statistics we're about to generate and shade with some fun colors just to take a look at it now (just because we want a table of values, doesn't mean we can't also visualize it if we like).

In [ ]:
from datashader.colors import Set1

counties_raster = cvs.polygons(county_df, geometry="geometry", agg=ds.max("OBJECTID"))
counties_image = shade(counties_raster, cmap=Set1, alpha=225, how="linear")
set_background(counties_image, "black")

Now, let's generate those statistics.

After a quick clean-up, we'll put our rasters in as arguments: we put counties as the zones, put target deserts as the values  
to segregate into those zones, and provide a dict with our desired statistics function to perform.

Note: `zonal_stats` comes with its own set of default calculations it will perform if the 3rd "zonal_funcs" argument is left empty, and please do feel free to clear that out and see what happens.
But, for our purposes, all we need is a simple mean calculation, so we'll set that into a dict, which we'll put into zonal_stats.



In [ ]:
from xrspatial import zonal_stats

# zones
counties_raster.data = counties_raster.data.astype(np.int64)

# values to summarize
target_deserts.data = target_deserts.data.astype(np.int8)

# summary functions
zonal_funcs = dict(pharmacy_desert_mean=lambda z: z.mean())

# execute summary functions on each zone and take top 10
results = zonal_stats(counties_raster, target_deserts, stats_funcs=zonal_funcs)

Now we can merge this results DataFrame back into the original counties one. 

In [ ]:
cols = ["pharmacy_desert_mean", "NAME", "STATE_FIPS", "geometry"]
final_df = pd.merge(county_df, results, left_on="ZONE_ID", right_index=True)[cols]
final_df.nlargest(10, "pharmacy_desert_mean")

Finally, we'll rasterize and shade these to visualize the results. We've highlighted the 100 counties with the highest pharmacy desert measures for seniors and shaded them by how much distance away, on average, the pharamcies are.

In [ ]:
from xrspatial import hillshade

counties_raster = cvs.polygons(county_df, geometry="geometry", agg=ds.max("OBJECTID"))

desert_raster = cvs.polygons(
    final_df.nlargest(75, "pharmacy_desert_mean"),
    geometry="geometry",
    agg=ds.mean("pharmacy_desert_mean"),
)

county_mask = cvs.polygons(county_df, geometry="geometry")

In [ ]:
img = stack(
    shade(county_mask, cmap=["#0f9425"], alpha=255),
    shade(counties_raster, cmap=["green", "#ffffff"], alpha=25),
    shade(desert_raster, cmap=["white", "#ff0000"], alpha=255),
    shade(hillshade(desert_raster), cmap=["green", "#ff0000"], alpha=150),
)
set_background(img, "black")